In [1]:
pip install opencv-python

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/seaborn-0.11.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import os

def generate_dataset():
    # Load the pre-trained face classifier
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Detect faces in the image
        faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) == 0:  # Check if no faces are detected
            return None

        # Assuming there's only one face, cropping the last detected face
        x, y, w, h = faces[-1]
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face

    cap = cv2.VideoCapture(0)  # Use the default camera (0)
    id = 1
    img_id = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to read frame from camera.")
            break

        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user{id}.{img_id}.jpg"  # File path format using f-string

            # Create directory if it doesn't exist
            if not os.path.exists("data"):
                os.makedirs("data")

            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)  # Fixed font parameter
            cv2.imshow("Cropped_face", face)

            if cv2.waitKey(1) == 13 or img_id == 100:  # Break condition
                break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting Samples Completed!")

generate_dataset()


Train the Classifier

In [3]:
import cv2
import numpy as np
from PIL import Image
import os

In [4]:
pip install opencv-contrib-python


DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/seaborn-0.11.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pillow

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/seaborn-0.11.2-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
cv2.face.LBPHFaceRecognizer_create() 

< cv2.face.LBPHFaceRecognizer 0x107a77fd0>

In [7]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        try:
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image)[1].split(".")[1])

            faces.append(imageNp)
            ids.append(id)
        except Exception as e:
            print(f"Error processing file {image}: {e}")

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")

train_classifier("data")


Error processing file data/.DS_Store: cannot identify image file '/Users/mba/Desktop/Face_Recognizer/data/.DS_Store'


Detect the Name and Face

In [8]:
import cv2
import numpy as np
from PIL import Image
import os

In [9]:
import cv2

# Load the classifier models
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

# Define a dictionary for storing the ages corresponding to each ID
ages = {
    1: 10,  # Example age for ID 1
    2: 12   # Example age for ID 2
}

# Open the default camera (index 0)
video_capture = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, img = video_capture.read()
    
    # Check if the frame was read successfully
    if not ret:
        print("Error: Failed to read frame from the camera")
        break
    
    # Process the frame
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray_img, scaleFactor=1.3, minNeighbors=6)
    
    for (x, y, w, h) in faces:
        # Draw bounding box around detected face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 2)
        
        # Recognize and label the face
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100*(1-pred/300))
        if confidence < 70:
            label = "UNKNOWN"
            age = "N/A"
        else:
            if id==1:
                label = "Ayesha"
                age = ages.get(id, ages[1])

            else:
                label = "Ruhma"
                age = ages.get(id, ages[2])
        
        # Display name and age
        display_text = f"{label}, {age}"
        cv2.putText(img, display_text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 1, cv2.LINE_AA)

    # Display the processed frame
    cv2.imshow("Face Detection", img)
    
    # Check for exit key
    if cv2.waitKey(1) == 13:  # Enter key
        break

# Release the camera and close OpenCV windows
video_capture.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 